# Experiment

## Pipelined processor

This file defines a neural network that processes several specrums individually. For each example, individual channels are sent through their own neural network. To use this system for classification, the output of all the neural networks must be combined with a post-processor. 

In this experiment we are processing 5 different channels: Visual red, visual green, visual blue, altitude data, and night-time near-IR data (city lights). The data was collected and saved as a TFRecord previously in the Data Aquisition chapter.

In [ ]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

# Math Stuff
import scipy.misc, random, os
import numpy as np

# Google Earth Engine
import ee
from gee_library import *
ee.Initialize()

# debug stuff
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Threads
import time, Queue
from tqdm import trange
from threading import Thread

# Tensorflow
import tensorflow as tf

from google.protobuf.internal import api_implementation
print(api_implementation._default_implementation_type)


# Globals

In [ ]:
DATA_DIR="data/experiment"
TEST_PROTO_FILENAME = os.path.join(DATA_DIR,"multi_spectrum_test.tfrecords")
TRAIN_PROTO_FILENAME = os.path.join(DATA_DIR,"multi_spectrum_train.tfrecords")

## Data Processing

The `get_batch()` function reads from the TFRecord file we created before and extracts a batch of examples. It delivers each channel separatly.

This function is hard-coded for the spectrums used in this experiment to maximize readibility.

In [ ]:
def get_batch(proto_filename, batch_size):

    NUMBER_OF_CLASSES=3
    
    filename_queue = tf.train.string_input_producer([proto_filename], num_epochs=None)
    proto_reader = tf.TFRecordReader()

    # get an example from file
    _, serialized_example = proto_reader.read(filename_queue)

    # unpack it
    features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'label': tf.FixedLenFeature([], tf.int64),
            'R': tf.FixedLenFeature([50*50], tf.int64),
            'G': tf.FixedLenFeature([50*50], tf.int64),
            'B': tf.FixedLenFeature([50*50], tf.int64),
            'elevation': tf.FixedLenFeature([50*50], tf.int64),
            'nightlights': tf.FixedLenFeature([50*50], tf.int64),
        })

    # now we have the raw data, but we need to convert it to 32-bit floats so
    # we can do some advance math on it.
    R = tf.cast(features['R'], tf.float32)
    G = tf.cast(features['G'], tf.float32)
    B = tf.cast(features['B'], tf.float32)
    elevation = tf.cast(features['elevation'], tf.float32)
    nightlights = tf.cast(features['nightlights'], tf.float32)
    
    # The label is an integer; convert it to a one-hot array
    label = tf.one_hot(features['label'], NUMBER_OF_CLASSES)

    # and batch it
    R_batch, G_batch, B_batch, elevation_batch, nightlight_batch, labels_batch = tf.train.shuffle_batch(
        [R, G, B, elevation, nightlights, label],
        batch_size=batch_size,
        capacity=2000,
        min_after_dequeue=1000)

    return R_batch, G_batch, B_batch, elevation_batch, nightlight_batch, labels_batch

## Network Definition

Now we will define the pipelined neural network. For simplicity, this example will use identical networks for each channel. Instead of writing out the network definition 5 times, though, we will define a function that takes data as input and returns the network output.

In [ ]:
#
# Required input placeholders
#
training = tf.placeholder(dtype=tf.bool, name="is_training") # True if training, False if testing
batch_size = tf.placeholder(dtype=tf.int32, name="batch_size")


##############
# Data Input #
##############

# Define variables for both the traning and test batches. We will only use one of them per iteration, though.
R_train_batch, G_train_batch, B_train_batch, elevation_train_batch, nightlight_train_batch, labels_train_batch = get_batch(TRAIN_PROTO_FILENAME, batch_size)
R_test_batch, G_test_batch, B_test_batch, elevation_test_batch, nightlight_test_batch, labels_test_batch = get_batch(TRAIN_PROTO_FILENAME, batch_size)

# Use a conditional tensor to select between using the test and training data
R_batch, G_batch, B_batch, elevation_batch, nightlight_batch, labels_batch = tf.cond(training,
                         lambda: (R_train_batch, G_train_batch, B_train_batch, elevation_train_batch, nightlight_train_batch, labels_train_batch), # training==True
                         lambda: (R_test_batch,  G_test_batch,  B_test_batch,  elevation_test_batch,  nightlight_test_batch,  labels_test_batch)# training==False
                        )



####################
# Model Definition #
####################

def pipelined_model(x, y_):

    # Batch Normalization
    input_norm = tf.contrib.layers.batch_norm(x, 
                                      center=True, scale=True, 
                                      is_training=training)



    # Resize the data from 2500 element vectors to 2D images.
    input_layer = tf.reshape(input_norm, [-1, 50, 50, 1])


    #
    # Convolution and Pooling Layers
    #

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[3, 3],
        padding="valid",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    # pool1_norm = tf.contrib.layers.batch_norm(pool1, center=True, scale=True, activation=tf.nn.relu, is_training=training)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[3, 3],
        padding="valid",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    # pool2_norm = tf.contrib.layers.batch_norm(pool2, center=True, scale=True, activation=tf.nn.relu, is_training=training)

    # Convolutional Layer #3 (no pooling)
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=64,
        kernel_size=[3, 3],
        padding="valid",
        activation=tf.nn.relu)
    # conv3_norm = tf.contrib.layers.batch_norm(conv3, center=True, scale=True, activation=tf.nn.relu, is_training=training)


    #
    # Fully Connected Layers
    #


    # Dense Layer
    conv3_flat = tf.reshape(conv3, [-1, 9 * 9 * 64])
    fc1 = tf.layers.dense(inputs=conv3_flat, units=1024, activation=tf.nn.relu)
    # fc1_norm = tf.contrib.layers.batch_norm(fc1, center=True, scale=True, activation=tf.nn.relu, is_training=training)
    dropout = tf.layers.dropout(
        inputs=fc1,
        rate=0.6,
        training= True)

    # Model Output. This is the output of the last dense layer; it still has not
    # had the "softmax" applied to it since the softmax cross-entropy loss tensor
    # will handle that transformation. So the value of every element is something
    # between positive and negative infinity.
    y = tf.layers.dense(inputs=dropout, units=3)



    #
    # Loss
    #

    loss = None
    train_op = None

    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=y_, # ground truth
        logits=y) # network output


    #
    # Accuracy Output; helpful for observing performance
    # 

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    model_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return y, loss, model_accuracy
    
    
    
#
# Now that we have defined the model as a function, we will feed
# each layer into it individually.
#

    # Here's where we stack the channels to create a 50x50x5 pixel example batch.
    x = tf.stack(values=[R_batch, G_batch, B_batch, elevation_batch, nightlight_batch],
                 axis=2)
    y_ = labels_batch


    
R_logits, R_loss, R_accuracy = pipelined_model(R_batch, labels_batch)
G_logits, G_loss, G_accuracy = pipelined_model(G_batch, labels_batch)
B_logits, B_loss, B_accuracy = pipelined_model(B_batch, labels_batch)
E_logits, E_loss, E_accuracy = pipelined_model(elevation_batch, labels_batch)
N_logits, N_loss, N_accuracy = pipelined_model(nightlight_batch, labels_batch)

# Softmax output. These tensors are what we will use for predictions.

R_y = tf.nn.softmax(R_logits)
G_y = tf.nn.softmax(G_logits)
B_y = tf.nn.softmax(B_logits)
E_y = tf.nn.softmax(E_logits)
N_y = tf.nn.softmax(N_logits)

In [ ]:
#
# Train
#

# Since we have 5 different pipelines, each processing different channels, we
# could process each individually, one after another. However, we can define
# an "overall" loss to minimize so that Tensorflow trains all 5 networks simultaneously.
# (If this is confusing, just know that tf.train.AdamOptimizer.minimize will adjust the
# weights of the network to minimize any variable. We *could* try to minimize R_loss,
# and then try to minimize G_loss, and then try to minimize B_loss etc. What I've done
# here is average all the losses together and asked tf.train.AdamOptimizer.minimize to
# minimize their average. The result is that it will try to minimize all the losses so that
# their average goes down.)
overall_loss = R_loss + G_loss + B_loss + E_loss + N_loss

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) # Required for batch norm
with tf.control_dependencies(update_ops): # Required for batch norm
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 1e-5
    learning_rate = tf.train.exponential_decay(learning_rate = starter_learning_rate,
                                               global_step = global_step,
                                               decay_steps = 200,
                                               decay_rate = 0.96,
                                               staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(overall_loss, global_step=global_step)


# Initialize TensorFlow
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# start the data reader tensors
tf.train.start_queue_runners(sess=sess)

# And run 20k iterations
for i in range(12000):
    if i%100 == 0:
        (loss_out,
        R_loss_out, G_loss_out, B_loss_out,
        E_loss_out, N_loss_out,
        R_accuracy_out, G_accuracy_out, B_accuracy_out,
        E_accuracy_out, N_accuracy_out) = sess.run( fetches = [overall_loss, R_loss, G_loss, B_loss, E_loss, N_loss,
                                                              R_accuracy, G_accuracy, B_accuracy, E_accuracy, N_accuracy],
                                                   feed_dict={training: False, batch_size:128})
        print ""
        print "-- Step", i, "--"
        print "- Loss Total:", loss_out
        print "   R:", R_loss_out, "G:", G_loss_out, "B:", B_loss_out, "Elevation:", E_loss_out, "Nightlights:", N_loss_out
        print "- Accuracy"
        print "   R:", R_accuracy_out, "G:", G_accuracy_out, "B:", B_accuracy_out, "Elevation:", E_accuracy_out, "Nightlights:", N_accuracy_out
        
        
    # run an iteration
    optimizer.run(feed_dict={batch_size:128,
                             training: True})


## Evaluation and Prediction

This experiment pipelined the analysis of each spectrum. Each pipeline is able to classify an example image using only the spectrum it is designed for. In machine learning, each of these pipelines might be refered to as "weak learners" since they each learn different aspects of a dataset. In order to create a "strong learner" we must combine the classifications for each pipeline to create an overall classification. There are many ways to do this.

The simplest way to combine weak learner classifications is to average the classifications of all the learners. A related way is to give each learner a vote and tally the results. A more sophisticated method would be to create another neural network that is given the outputs of each weak learner and some information about the input data. This last method could actually learn which classifier to trust more depending on what sort of image was given.

But to keep it simple we will show the output of each weak learner, as well as the average of all the classifications.

In [ ]:
#
# Evaluate
#


(R_accuracy_out,
 G_accuracy_out,
 B_accuracy_out,
 E_accuracy_out,
 N_accuracy_out) = sess.run( fetches = [R_accuracy,
                                        G_accuracy,
                                        B_accuracy,
                                        E_accuracy,
                                        N_accuracy],
                             feed_dict={training: False, batch_size:128})
print "Accuracy in the test set:"
print "   R:", R_accuracy_out, "G:", G_accuracy_out, "B:", B_accuracy_out, "Elevation:", E_accuracy_out, "Nightlights:", N_accuracy_out

In [ ]:
#
# Prediction
#
# Remember, the classes are:
# farm:     0  one-hot: [1,0,0]
# city:     1  one-hot: [0,1,0]
# mountain: 2  one-hot: [0,0,1]
#
# You can keep rerunning this cell to test more images (CTRL-Enter runs the selected cell.)



(R_batch_out,
 G_batch_out,
 B_batch_out,
 E_batch_out,
 N_batch_out,
 labels_out,
 R_y_out,
 G_y_out,
 B_y_out,
 E_y_out,
 N_y_out) = sess.run( fetches = [R_batch,
                                 G_batch,
                                 B_batch,
                                 elevation_batch,
                                 nightlight_batch,
                                 labels_batch,
                                 R_y,
                                 G_y,
                                 B_y,
                                 E_y,
                                 N_y],
                      feed_dict={training: False, batch_size:1})

print "Red channel:"
plt.imshow(R_batch_out[0,:].reshape((50,50)), cmap='gray'); plt.show()
print "Green channel:"
plt.imshow(G_batch_out[0,:].reshape((50,50)), cmap='gray'); plt.show()
print "Blue channel:"
plt.imshow(B_batch_out[0,:].reshape((50,50)), cmap='gray'); plt.show()
print "Elevation channel:"
plt.imshow(E_batch_out[0,:].reshape((50,50)), cmap='gray'); plt.show()
print "Night Light channel:"
plt.imshow(N_batch_out[0,:].reshape((50,50)), cmap='gray'); plt.show()

print "Ground truth Label:", labels_out, "Max:", np.argmax(labels_out)
print "Predicted labels:"
print "R:", R_y_out, "Max:", np.argmax(R_y_out)
print "G:", G_y_out, "Max:", np.argmax(G_y_out)
print "B:", B_y_out, "Max:", np.argmax(B_y_out)
print "E:", E_y_out, "Max:", np.argmax(E_y_out)
print "N:", N_y_out, "Max:", np.argmax(N_y_out)
